# Excercise 4:
# Laplace Approximation of Logistic Regression
### presented by: Juan David Gil and Juan Sebastián Silva
### Date: 29 february 2016

In [62]:
% pylab inline
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d.axes3d import Axes3D
import pylab as pb
from matplotlib import cm
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [117]:
cases = []

file_name = 'data.csv'
X = np.loadtxt(open(file_name,'rb'),delimiter=',')
cases.append(X[:,1:X.shape[1]])
##print X[:,1:X.shape[1]-1]
## si son sanos, cases[0][x,16]=0
##si son enfermos, cases[0][x,16]=1
XX = X[:,1:]
park = np.zeros(XX.shape[1])
nopark = np.zeros(XX.shape[1])
for i in range(0, XX.shape[0]):
    mclass = XX[i,16]
    if(mclass == 1.0):
        park = np.vstack((park, XX[i,:]))
    else:
        nopark = np.vstack((nopark, XX[i,:]))

idx = np.ones(nopark.shape[0], dtype=bool)
idx[0] = False
nopark = nopark[idx,:]
idx = np.ones(park.shape[0], dtype=bool)
idx[0] = False
park = park[idx,:]
XX = np.vstack((nopark,park))

#idx = np.zeros(XX.shape[1], dtype=bool)
#idx[16] = True
#T = XX[:,idx]

T = np.zeros((XX.shape[0],2))
for i in range(0,XX.shape[0]):
    T[i,XX[i,16]] = 1.0

#print T
idx = np.ones(XX.shape[1], dtype=bool)
idx[16] = False
XX = XX[:,idx]
#print XX[:,16]
print XX.shape, T.shape

(195, 22) (195, 2)


In [127]:
# Logistic function
def logisFunc(a):
    ret = 1.0/(1.0 + np.exp(-a))
    return ret

    # Extracting the class wich probability is the highest
def extractMax(Yestim, nclass):
    for i in range(len(Yestim[:,0])):
        mayor = -1000000.0
        idmayor = -1
        for j in range(nclass):
            if(Yestim[i,j] > mayor):
                mayor = Yestim[i,j]
                idmayor = j

        for j in range(nclass):
            if (j == idmayor):
                Yestim[i,j] = 1.
            else:
                Yestim[i,j] = 0.
    return Yestim

    # Evaluating accuracy of a preditcion
def evalAccuracy(Yestim, Tvalid, nclass):
    true_positives = np.zeros(nclass)
    true_negatives = np.zeros(nclass)
    false_positives = np.zeros(nclass)
    false_negatives = np.zeros(nclass)
    for i in range(len(Yestim[:,0])):
        for j in range(nclass):
            if(Tvalid[i,j] == 1. and Tvalid[i,j] == Yestim[i,j]):
                true_positives[j] += 1.
            if(Tvalid[i,j] == 0. and Tvalid[i,j] == Yestim[i,j]):
                true_negatives[j] += 1.
            if(Tvalid[i,j] == 0. and Yestim[i,j] == 1.):
                false_positives[j] += 1.
            if(Tvalid[i,j] == 1. and Yestim[i,j] == 0):
                false_negatives[j] += 1.

    ret = np.sum(true_positives + true_negatives)/np.sum(true_positives + false_positives + false_negatives + true_negatives)
    ret_class1 = (true_positives[0] + true_negatives[0])/(true_positives[0] + false_positives[0] + false_negatives[0] + true_negatives[0])
    return ret

def meanDesvCacl(z):
    mmean = np.sum(z)/len(z)
    desv = np.sum((z - mmean)*(z - mmean))/len(z)
    return mmean, desv

def logRegEstim(X,T, Xtest):
    PHI = X
    N = X.shape[0]
    M = X.shape[1]
    #Wold = np.transpose(np.matrix(np.random.random_sample(M)))
    Wold = np.transpose(np.matrix(np.zeros(M))) + 0.001
    Wnew = np.transpose(np.matrix(np.zeros(M)))
    EPS = 1e-6
    diff = np.linalg.norm(Wold - Wnew)
    cont = 0
    Yestim = T
    var_0 = 10.0
    S0 = np.eye(M)*var_0
    mean_0 = np.zeros(M)
    
    S0 = np.matrix(S0)
    mean_0 = mean_0[:,None]
    mean_0 = np.matrix(mean_0)
    
    # Newton raphson with several iteration
    super_R = 0
    while cont < 100:
        yy = np.transpose(Wold) * np.transpose(PHI)
        yy = np.transpose(logisFunc(yy))
        yy = yy
        Yestim = yy        
        #print ' '
        R = np.zeros((N,N))
        for i in range(0,N):
            #print yy[i,0]
            R[i,i] = (yy[i,0] * (1 - yy[i,0])) + 0.001
        super_R = R
        #z = (PHI*Wold) - (np.linalg.inv(R)*(yy - T))
        #Wnew = np.linalg.inv((np.transpose(PHI) * (R * PHI))) * (np.transpose(PHI)*R*z)
        tmp1 = (np.linalg.inv(S0) + np.transpose(PHI)*(R*PHI))
        tmp2 = (np.linalg.inv(S0)*(Wold - mean_0) + np.transpose(PHI)*(yy - T))
        #print tmp1.shape, tmp2.shape
        Wnew = Wold - (np.linalg.inv(tmp1)*(tmp2))
        diff = np.linalg.norm(Wold - Wnew)
        if diff < EPS:
            break
        else:
            Wold = Wnew
        cont += 1
    #print Wnew
    cov_mat = np.linalg.inv(S0) +   np.transpose(PHI)*(super_R*PHI)
    #print Xtest.shape
    ret = np.transpose(Wnew) * np.transpose(Xtest)
    ret = np.transpose(logisFunc(ret))
    #print ret.shape
    return ret

# Cross validation -----------------------------------------------------------
def evaluate(Xtrain, Xtest, Ttrain, Ttest, nclass):
    
    # logistic regression
    Yestim_log_reg = np.zeros((len(Ttest[:,0]), nclass))
    Yestim_log_reg = np.matrix(Yestim_log_reg)
    for i in range(0, nclass):
        Yestim_log_reg[:,i] = logRegEstim(Xtrain, Ttrain[:,i], Xtest)
    Yestim_log_reg = extractMax(Yestim_log_reg,nclass)
    log_reg_err = evalAccuracy(Yestim_log_reg, Ttest, nclass)

    return log_reg_err


def crossValidation(X, T, nclass, valid_num = 5):
    N = len(X[:, 0])
    Nvalid = (N/valid_num)
    Ntest = N - Nvalid
    # Setting a seed for random elections, comment this line if you want a new permutation in each execution of this subroutine
    np.random.seed(3)
    # Generating a random permutation of the indexes of the array
    index = np.random.permutation(N)
    l = 0
    r = Nvalid
    # Arrays to save the values of the accuracy obtained in each  validation
    #err_lse_arr = np.zeros(valid_num)
    #err_gme_arr = np.zeros(valid_num)
    err_log_arr = np.zeros(valid_num)    
    for i in range(0, valid_num):
        # Dividing the training groups of the validation group for each validation
        Xtrain = []
        Ttrain = []
        if l == 0:
            Xtrain = X[index[r:N], :]
            Ttrain = T[index[r:N], :]
        else:
            if(r < N):
                Xtrain = np.concatenate((X[index[0:l], :],X[index[r: N], :]))
                Ttrain = np.concatenate((T[index[0:l], :],T[index[r: N], :]))
            else:
                Xtrain = X[index[0:l], :]
                Ttrain = T[index[0:l], :]


        Xvalid = X[index[l:r], :]
        Tvalid = T[index[l:r], :]
        l = r
        r += Nvalid
        '''
        print Xvalid.shape
        print Tvalid.shape
        print Xtrain.shape
        print Ttrain.shape
        print ' '
        '''
        # calculating the accuracy for the ith validation for each method
        #print Xtrain.shape, Xvalid.shape, Ttrain.shape, Tvalid.shape
        err_log_arr[i] = evaluate(Xtrain, Xvalid, Ttrain, Tvalid, nclass)
        
    print 'Accuracy logistic regression classification: '
    #print err_log_arr
    acc, desv = meanDesvCacl(err_log_arr)
    print 'Accuracy: ', ("%.3f" % acc), ' Standard Deviation', ("%.3f" % desv), ' Maximum accuracy: ', ("%.3f" % np.amax(err_log_arr))
    print '---------------------------------------------------------------'

In [128]:
crossValidation(XX, T, 2)

Accuracy logistic regression classification: 
Accuracy:  0.841  Standard Deviation 0.004  Maximum accuracy:  0.923
---------------------------------------------------------------
